In [1]:
from pathlib import Path
import numpy as np
import tifffile
from napari_flim_phasor_plotter._reader import read_stack

In [2]:
zarr_stack_path = Path(r"D:\Datasets\FLIM\Lifetime Unmixing Project\TMR31_3_sptw\TMR31_3_ptu\TMR31_3_ptu.zarr")
zarr_stack_path = Path(r"D:\Datasets\FLIM\ptu_files\z_stack\stack_smaller\stack_smaller.zarr")

pixel_size_x = 0.2696
pixel_size_y = 0.2696
pixel_size_z = 1
pixel_size_unit = 'µm'
u_time_resolution = 12.5
u_time_unit = 'ps'
channel_names = ['Centrosomes', 'Membrane and Chromatin']

In [3]:
zarr_stack_path.parent

WindowsPath('D:/Datasets/FLIM/ptu_files/z_stack/stack_smaller')

In [4]:
stack, metadata = read_stack(zarr_stack_path)

stack = True 
data type:  .zarr 
data_shape =  (2, 136, 1, 44, 256, 256) 



Save flim dimension as if it were time. One time point per file. it seems to work with zarr, but zarr file has no metadata, so it must be added manually before saving.

In [7]:
# Define the metadata for OME-TIFF
ome_metadata = {
    'axes': 'CTZYX',
    'PhysicalSizeX': pixel_size_x,
    'PhysicalSizeXUnit': pixel_size_unit,
    'PhysicalSizeY': pixel_size_y,
    'PhysicalSizeYUnit': pixel_size_unit,
    'PhysicalSizeZ': pixel_size_z,
    'PhysicalSizeZUnit': pixel_size_unit,
    'TimeIncrement': u_time_resolution,
    'TimeIncrementUnit': u_time_unit,
    'Channel': {'Name': [f'{channel}' for channel in channel_names]},
}

output_dir = zarr_stack_path.parent / 'OME_TIF'
output_dir.mkdir(exist_ok=True, parents=True)

for t in range(stack.shape[2]):
    print(t)
    # Save the image as an OME-TIFF with lossless compression
    if stack.shape[2] == 1:
        output_file_name = zarr_stack_path.stem + '.ome.tif'
        ome_metadata['Name'] = zarr_stack_path.stem
    else:
        # get time with leading zeros
        t_string = str(t).zfill(len(str(stack.shape[2])))
        output_file_name = zarr_stack_path.stem + f'_t{t_string}.ome.tif'
        ome_metadata['Name'] = zarr_stack_path.stem + f'_t{t_string}'
    output_path = output_dir / output_file_name
    with tifffile.TiffWriter(output_path, ome=True) as tif:
        tif.write(stack[:,:,t], metadata=ome_metadata, compression='zlib')
    # if t==0:
    #     break

0


KeyboardInterrupt: 